In [24]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 3.9MB/s 
     |████████████████████████████████| 1.1MB 20.5MB/s 
     |████████████████████████████████| 890kB 31.7MB/s 
     |████████████████████████████████| 3.0MB 41.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=47d8fac46cf668c9d1152112341ed15e8d43cbed4d5f60979cde1433613f3dab
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import transformers
import numpy as np
import json
import pandas as pd
import re
import io

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import sklearn.metrics as metrics

In [6]:
nlp_bert = transformers.TFBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFBertModel: ['vocab_projector', 'vocab_layer_norm', 'distilbert', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

# Generating META features

In [8]:
def get_metadata_features(i):
    print("Inside get_metadata_features")
    """
    Obtain the dataset
    """
    # Extracted the text for nlp embeddings  
    df_X_train= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_train_' + str(i) + '.pkl')
    #print(df_X_train.head())
    df_X_test= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_test_' + str(i) + '.pkl')
    #print(df_X_train.shape) #(835, 5)
    #print(df_Y_train.shape) #(835, 1)
    #print(df_X_test.shape) #(209, 5)
    #print(df_Y_test.shape) #(209, 1)
    
    X_train_meta = df_X_train.drop(['Previous_User_Utterance'], axis=1)
    X_test_meta = df_X_test.drop(['Previous_User_Utterance'], axis=1)
    
    #print(X_train_meta['Previous_Speech_Act'].unique())
    #print(X_train_meta['Previous_Search_Act'].unique())
    
    return X_train_meta, X_test_meta, df_Y_train, df_Y_test

# Generating BERT features

In [9]:
def load_files(i):

    df_X_train= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_train_' + str(i) + '.pkl')
    df_X_test= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_test_' + str(i) + '.pkl')
    df_train = pd.concat([df_X_train,df_Y_train], axis=1)
    df_test = pd.concat([df_X_test,df_Y_test], axis=1)

    #print(list(df_train.columns))

    df_train = df_train[["Search_acts","Previous_User_Utterance"]]
    df_train = df_train.rename(columns={"Search_acts":"y", "Previous_User_Utterance":"text"})

    

    df_test = df_test[["Search_acts","Previous_User_Utterance"]]
    df_test = df_test.rename(columns={"Search_acts":"y", "Previous_User_Utterance":"text"})

    return df_train,df_test,df_Y_train,df_Y_test

In [10]:
def generate_maskid(training, test):
  #print(training['text'])
  corpus_train = training['text']
  corpus_test = test['text']
  #the length of the feature vector is 150
  maxlen = 150

  #add special tokens
  maxqnans = np.int((maxlen-20)/2)
  corpus_tokenized_train = ["[CLS] "+
              " ".join(tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', 
              str(txt).lower().strip()))[:maxqnans])+
              " [SEP] " for txt in corpus_train]
  corpus_tokenized_test = ["[CLS] "+
              " ".join(tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', 
              str(txt).lower().strip()))[:maxqnans])+
              " [SEP] " for txt in corpus_test]
  #generate masks
  masks_train = [[1]*len(txt.split(" ")) + [0]*(maxlen - len(txt.split(" "))) for txt in corpus_tokenized_train]
  masks_test = [[1]*len(txt.split(" ")) + [0]*(maxlen - len(txt.split(" "))) for txt in corpus_tokenized_test]

  #padding
  txt2seq_train = [txt + " [PAD]"*(maxlen-len(txt.split(" "))-2) if len(txt.split(" ")) != (maxlen) else txt for txt in corpus_tokenized_train]
  txt2seq_test = [txt + " [PAD]"*(maxlen-len(txt.split(" "))-2) if len(txt.split(" ")) != (maxlen) else txt for txt in corpus_tokenized_test]

  #generate idx
  idx_train = [tokenizer.encode(seq.split(" ")) for seq in txt2seq_train]
  idx_test = [tokenizer.encode(seq.split(" ")) for seq in txt2seq_test]      

  ## feature matrix
  X_train = [np.asarray(idx_train, dtype='int32'), 
            np.asarray(masks_train, dtype='int32')]
  X_test = [np.asarray(idx_test, dtype='int32'), 
            np.asarray(masks_test, dtype='int32')]
  

  idx_train = np.asarray(idx_train, dtype='int32')
  masks_train = np.asarray(masks_train, dtype='int32')
  idx_test = np.asarray(idx_test, dtype='int32')
  masks_test = np.asarray(masks_test, dtype='int32')

  #print("txt: ", training["text"].iloc[0])
  #print("tokenized:", [tokenizer.convert_ids_to_tokens(idx) for idx in X_train[0][i].tolist()])
  #print("idx: ", X_train[0][i])
  #print("mask: ", X_train[1][i])
  return idx_train, masks_train, idx_test, masks_test


In [11]:
def model_generate(y_train):

  ## inputs
  idx = layers.Input((150), dtype="int32", name="input_idx")
  masks = layers.Input((150), dtype="int32", name="input_masks")
  ## pre-trained bert with config
  config = transformers.DistilBertConfig(dropout=0.2,attention_dropout=0.2)
  config.output_hidden_states = False
  nlp_bert = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)
  bert_out = nlp_bert(idx, attention_mask=masks)[0]
  ## fine-tuning
  x = layers.GlobalAveragePooling1D()(bert_out)
  x = layers.Dense(64, activation="relu")(x)
  #x = layers.Dense(32, activation="relu")(x)
  y_out = layers.Dense(len(np.unique(y_train)), activation='softmax')(x)

  #nlp2 = transformers.TFDistilBertModel.from_pretrained('distilroberta-base', config=config)
  #bert_out2 = nlp2(idx, attention_mask=masks)[0]
  #x2 = layers.GlobalAveragePooling1D()(bert_out2)
  #x2 = layers.Dense(32, activation="relu")(x2)
  #y_out = layers.Dense(len(np.unique(y_train)), activation='softmax')(x2)



  ## compile
  model = models.Model([idx, masks], y_out)
  for layer in model.layers[:3]:
      layer.trainable = False
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()

  return model

In [12]:
def model_train(X_train, y_train, model):

  ## encode y
  dic_y_mapping = {n:label for n,label in enumerate(np.unique(y_train))}
  print(dic_y_mapping)
  inverse_dic = {v:k for k,v in dic_y_mapping.items()}
  print(inverse_dic)
  y_train2 = np.array([inverse_dic[y] for y in y_train])
  ## train
  training = model.fit(x=X_train, y=y_train2, batch_size=32, epochs=300, shuffle=True, verbose=1, validation_split=0.3)

  training_acc = training.history['accuracy']
  training_loss = training.history['loss']
  #print(acc)
  #loss, accuracy = model.evaluate(x=X_test, y=y_test, batch_size=32, verbose=1)
  return training_acc, training_loss
  #loss, accuracy = model.evaluate(x=X_train, y=y_train, batch_size=32, verbose=1)

In [13]:
def test_classification(X_test, y_test, y_train):

  dic_y_mapping = {n:label for n,label in enumerate(np.unique(y_train))}
  #print(np.unique(y_test))
  predicted_prob = model.predict(X_test)
  #print(predicted_prob)
  predicted = [dic_y_mapping[np.argmax(pred)] for pred in predicted_prob]

  return predicted, predicted_prob

# Generating NLP features

In [14]:
def encode_nlp_features(df, name, i):
    print("Inside encode_nlp_features")
    mlb = MultiLabelBinarizer()
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("ne")), columns=mlb.classes_, index=df.index).add_prefix('ne_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("iob")), columns=mlb.classes_, index=df.index).add_prefix('iob_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("alpha")), columns=mlb.classes_, index=df.index).add_prefix('al_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("digit")), columns=mlb.classes_, index=df.index).add_prefix('dig_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("punc")), columns=mlb.classes_, index=df.index).add_prefix('punc_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("url")), columns=mlb.classes_, index=df.index).add_prefix('url_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("oov")), columns=mlb.classes_, index=df.index).add_prefix('oov_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("stop")), columns=mlb.classes_, index=df.index).add_prefix('stop_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("coarsepos")), columns=mlb.classes_, index=df.index).add_prefix('cpos_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("finepos")), columns=mlb.classes_, index=df.index).add_prefix('fpos_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("dep")), columns=mlb.classes_, index=df.index).add_prefix('dep_'))], axis=1)
    #print(df.shape)
    df = pd.concat([df,(pd.DataFrame(mlb.fit_transform(df.pop("offset")), columns=mlb.classes_, index=df.index).add_prefix('off_'))], axis=1)
    #print(df.shape)
    #print(df.columns.values.tolist())
    df.to_pickle('output\\nlp-meta-attn\\df_' + name + '_nlp_features_' + str(i) + '.pkl')
    return df

In [15]:
"""
This method should generate all the natural language processing features
"""
def generate_nlp_features(df, i, name=""):
    print("Inside generate_nlp_features")
    sentence_ne = []
    sentence_iob = []
    sentence_alpha = []
    sentence_isdigit = []
    sentence_ispunc = []
    sentence_isurl = []
    sentence_isoov = []
    sentence_isstop = []
    sentence_coarsepos = []
    sentence_finepos = []
    sentence_dep = []
    sentence_offset = []

    i=0
   
    """NEED TO CHANGE THE FUNCTIONAL COLUMN"""
    for entries in df['Previous_User_Utterance']: #each entry is a sentence
        if i%100==0:
            print(i)
        i=i+1

        doc = nlp(entries) #Spacy function to get tags
        #print(doc)

        ne = []
        iob = []
        alpha = []
        isdigit = []
        ispunc = []
        isurl = []
        isoov = []
        isstop = []
        coarsepos = []
        finepos = []
        dep = []
        offset = []

        """
        For words in the sentence
        """
        for token in doc:
            """
            Named Entity Type
            """
            #print("NE Type:", token.ent_type)
            ne.append(token.ent_type_)

            """
            IOB code of named entity tag. B means the token begins an entity, 
            I means it is inside an entity, O means it is outside an entity, and 
            '' means no entity tag is set.
            """
            #print("IOB:", token.ent_iob) 
            iob.append(token.ent_iob_)

            """
            Does the token contain alphabetic characters?
            """
            #print("Alpha:", token.is_alpha)
            alpha.append(str(token.is_alpha))

            """
            Does the token contain digits?
            """
            #print("Digits:", token.is_digit)
            isdigit.append(str(token.is_digit))

            """
            Is the token punctuation?
            """
            #print("Punc:", token.is_punct)
            ispunc.append(str(token.is_punct))

            """
            Is the token like urls?
            """
            #print("Url:", token.like_url)
            isurl.append(str(token.like_url))

            """
            Is the token out-of-vocabulary?
            """
            #print("OOV:", token.is_oov)
            isoov.append(str(token.is_oov))

            """
            Is the token a stopword?
            """
            #print("Stop:", token.is_stop)
            isstop.append(str(token.is_stop))

            """
            Coarse-grained POS
            """
            #print("Coarse POS:", token.pos_)
            coarsepos.append(token.pos_)

            """
            Fine-grained POS
            """
            #print("Fine POS:", token.tag_)
            finepos.append(token.tag_)

            """
            Syntactic Dependency Relation
            """
            #print("Dependency:", token.dep_)
            dep.append(token.dep_)

            """
            Character offset
            """
            #print("Offset:", token.idx)
            coff = token.idx
            if(coff<100):
                off = 1
            elif(coff<200): 
                off =2
            elif(coff<300): 
                off =3
            elif(coff<400): 
                off =4
            elif(coff<500): 
                off =5
            elif(coff<600): 
                off =6
            elif(coff<700): 
                off =7
            elif(coff<800): 
                off =8
            elif(coff<900): 
                off =9
            elif(coff<1000): 
                off =10
            else: 
                off =11
            offset.append(off)
            
            #print(type(token.is_alpha))
            #break

        sentence_ne.append(ne)
        sentence_iob.append(iob)
        sentence_alpha.append(alpha)
        sentence_isdigit.append(isdigit)
        sentence_ispunc.append(ispunc)
        sentence_isurl.append(isurl)
        sentence_isoov.append(isoov)
        sentence_isstop.append(isstop)
        sentence_coarsepos.append(coarsepos)
        sentence_finepos.append(finepos)
        sentence_dep.append(dep)
        sentence_offset.append(offset)
    df["ne"] = sentence_ne
    df["iob"] = sentence_iob
    df["alpha"] = sentence_alpha
    df["digit"] = sentence_isdigit
    df["punc"] = sentence_ispunc
    df["url"] = sentence_isurl
    df["oov"] = sentence_isoov
    df["stop"] = sentence_isstop
    df["coarsepos"] = sentence_coarsepos
    df["finepos"] = sentence_finepos
    df["dep"] = sentence_dep
    df["offset"] = sentence_offset
    
    print(len(sentence_ne) == len(sentence_iob) == len(sentence_alpha) == len(sentence_isdigit) == \
          len(sentence_ispunc) == len(sentence_isurl) == len(sentence_isoov) == len(sentence_isstop) \
          == len(sentence_coarsepos) == len(sentence_finepos) == len(sentence_dep) == len(sentence_offset))

    df.to_pickle('output\\nlp-meta-attn\\df_' + name + '_nlp_features_intermediate_' + str(i) + '.pkl')
    return df

In [16]:
def get_nlp_encodings(i):
    """
    Obtain the dataset
    """
    print("Inside get_nlp_encodings")
    # Extracted the text for nlp embeddings  
    df_X_train= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_train_' + str(i) + '.pkl')
    df_X_test= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_test_' + str(i) + '.pkl')
    #print(df_X_train.shape) #(835, 5)
    #print(df_Y_train.shape) #(835, 1)
    #print(df_X_test.shape) #(209, 5)
    #print(df_Y_test.shape) #(209, 1)
    
    """
    merging the output labels to predictors
    """
    df_train = pd.concat([df_X_train,df_Y_train], axis=1)
    df_test = pd.concat([df_X_test,df_Y_test], axis=1)
    print("The size of training and test sets:", df_train.shape, df_test.shape)
    r,c = df_train.shape
    #print("Rows in training data =", r)
    """
    merging training and test data to one dataframe
    """
    df = pd.concat([df_train,df_test])
    #print("Final df:", df.shape)
    #print(df.head(2))
    """
    generating the nlp features using spacy
    """
    df_nlp = generate_nlp_features(df, i, 'all')
    #print("After-spacy:", df_nlp.shape, df_nlp.columns.values)
    #print(df_nlp.head(2))
    
    """
    encoding the nlp features using multilabelbinarizer
    """
    df_nlp_enc = encode_nlp_features(df_nlp, 'all_nlp_enc', i)
    #print(df_nlp_enc.shape)
    #print(df_nlp_enc.head(2))
    #print(df_nlp_enc.shape, df_nlp_enc.columns.values.tolist())
    df_nlp_enc.drop(['Utterance_Number', 'Duration', 'Speaker', 'System_Number', 'Search_Task', 'Previous_User_Utterance', 'Previous_User_Speech_Act', 'Previous_Speech_Act', 'Previous_Search_Act','Search_acts'], axis=1, inplace=True)
                     #['Transcript', 'Query_counter', 'Length', 'If_Intermediary', 'Complexity', 'Speech_acts'], axis=1, inplace=True)
    #print(df_nlp_enc.shape, df_nlp_enc.columns.values.tolist())
    train_split = df_nlp_enc.iloc[:r,:]
    test_split = df_nlp_enc.iloc[r:,:]
    print("Size after split:",train_split.shape, test_split.shape, df_Y_train.shape, df_Y_test.shape)
    return train_split, test_split, df_Y_train, df_Y_test

# The Attention Layer

In [17]:
!pip install Layer

In [18]:
from keras.engine.topology import Layer
class AttentionLayer(Layer):
   
    def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.trainable_weight = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

# DNN model creation and execution

In [19]:
def create_bilstm_meta_nlp_bert_channels(reshaped_data_nlp, reshaped_data_meta):
    
    idx = Input((150), dtype="int32", name="input_idx")
    masks = Input((150), dtype="int32", name="input_masks")
    ## pre-trained bert with config
    config = transformers.DistilBertConfig(dropout=0.2,attention_dropout=0.2)
    config.output_hidden_states = False
    nlp_bert = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)
    bert_out = nlp_bert(idx, attention_mask=masks)[0]
    ## fine-tuning
    x = GlobalAveragePooling1D()(bert_out)
    x = Dense(64, activation="relu")(x)
    #x = layers.Dense(32, activation="relu")(x)
    y_out = Dense(4, activation='softmax')(x)
    
    
    
    samples,timesteps,features = reshaped_data_nlp.shape
    inp_nlp = Input(shape=(timesteps,features), name='input_nlp')
    # lstm needs (samples,timesteps,features) tensor as the input
    x1 = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(inp_nlp) 
    x1, sent_coeffs1 = AttentionLayer(features,return_coefficients=True,name='sent_attention1')(x1)
    #x1 = GlobalMaxPool1D()(x1)
    #x1 = Dense(100, activation="relu")(x1)
    x1 = Dropout(0.25)(x1)
    x1 = Dense(4, activation="softmax")(x1) #output layer  
    
    samples,timesteps,features = reshaped_data_meta.shape
    inp_meta = Input(shape=(timesteps,features), name='input_meta')
    # lstm needs (samples,timesteps,features) tensor as the input
    x2 = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(inp_meta) 
    x2, sent_coeffs2 = AttentionLayer(features,return_coefficients=True,name='sent_attention2')(x2)
    #x1 = GlobalMaxPool1D()(x1)
    #x1 = Dense(100, activation="relu")(x1)
    x2 = Dropout(0.25)(x2)
    x2 = Dense(4, activation="softmax")(x2) #output layer  
      
    
    """
    Merging outputs of nlp and meta
    """
    merge_nlp_bert = concatenate([x1,x2,y_out])


    output_meta_nlp_bert = Dense(4, activation="softmax")(merge_nlp_bert)
    model_meta_nlp_bert_combined = Model(inputs=[inp_nlp, inp_meta, idx, masks], outputs=output_meta_nlp_bert)
    
    return model_meta_nlp_bert_combined

In [20]:
def execute_bilstm_meta_nlp_bert_channel(i):
    """
    nlp data
    """
    df_train_nlp_encodings, df_test_nlp_encodings, df_Y_train_nlp, df_Y_test_nlp = get_nlp_encodings(i)
    train_nlp_enc = df_train_nlp_encodings.values
    test_nlp_enc = df_test_nlp_encodings.values
    
    
    timestamp = 1 #number of successive sequences combined
    
    r, c = df_train_nlp_encodings.shape
    staggering = r - timestamp + 1 # final number of instances generated 
    X_train_reshaped_nlp = np.concatenate([train_nlp_enc[x:x+timestamp,:] for x in range(r-timestamp+1)])
    X_train_reshaped_nlp = X_train_reshaped_nlp.reshape(staggering, timestamp, c) # c is the number of features
    
    print("NLP data\n------------------")
    print("Training set\n------------------")
    print(X_train_reshaped_nlp)
    print(X_train_reshaped_nlp.shape)
    
    r2, c2 = df_test_nlp_encodings.shape
    staggering2 = r2 - timestamp + 1 # final number of instances generated 
    X_test_reshaped_nlp = np.concatenate([test_nlp_enc[x:x+timestamp,:] for x in range(r2-timestamp+1)])
    X_test_reshaped_nlp = X_test_reshaped_nlp.reshape(staggering2, timestamp, c2) # c is the number of features
    
    print("NLP data\n------------------")
    print("Test set\n------------------")
    print(X_test_reshaped_nlp)
    print(X_test_reshaped_nlp.shape)
    
    df_Y_train_nlp = df_Y_train_nlp.iloc[timestamp-1:,]
    df_Y_test_nlp = df_Y_test_nlp.iloc[timestamp-1:,]
    
    print("Output Labels\n------------------")
    print(df_Y_train_nlp.shape, df_Y_test_nlp.shape)    
    
    


    """
    meta data
    """
    df_train_meta, df_test_meta, df_Y_train_meta, df_Y_test_meta = get_metadata_features(i)
    train_meta = df_train_meta.values
    test_meta = df_test_meta.values
        
    timestamp = 1 #number of successive sequences combined
        
    r, c = df_train_meta.shape
    staggering = r - timestamp + 1 # final number of instances generated 
    X_train_reshaped_meta = np.concatenate([train_meta[x:x+timestamp,:] for x in range(r-timestamp+1)])
    X_train_reshaped_meta = X_train_reshaped_meta.reshape(staggering, timestamp, c) # c is the number of features
      
    r2, c2 = df_test_meta.shape
    staggering2 = r2 - timestamp + 1 # final number of instances generated 
    X_test_reshaped_meta = np.concatenate([test_meta[x:x+timestamp,:] for x in range(r2-timestamp+1)])
    X_test_reshaped_meta = X_test_reshaped_meta.reshape(staggering2, timestamp, c2) # c is the number of features
        
    print("Meta data\n------------------")
    print("Training set\n------------------")
    print(X_train_reshaped_meta)
    print(X_train_reshaped_meta.shape)
        
    print("Meta data\n------------------")
    print("Test set\n------------------")
    print(X_test_reshaped_meta)
    print(X_test_reshaped_meta.shape)
        
    df_Y_train_meta = df_Y_train_meta.iloc[timestamp-1:,]
    df_Y_test_meta = df_Y_test_meta.iloc[timestamp-1:,]
    
    
    print("Output Labels\n------------------")
    print(df_Y_train_meta.shape, df_Y_test_meta.shape)

    
    # ----------------------------------------------------
    # can save and read back for iterations
    # ----------------------------------------------------
    
    """
    bert data
    """

    df_train_bert,df_test_bert,df_Y_train_bert,df_Y_test_bert = load_files(i)
    y_train_bert = df_Y_train_bert['Search_acts'].to_list()
    y_test_bert = df_Y_test_bert['Search_acts'].to_list()
    X_train_id, X_train_mask, X_test_id, X_test_mask = generate_maskid(df_train_bert,df_test_bert)
    
    
    """
    $$$$ ---------------
    $$$$ CHANGE HERE _word_meta _word_nlp _nlp_meta _word_nlp_meta
    $$$$ ---------------
    """
    
    model_meta_nlp_bert = create_bilstm_meta_nlp_bert_channels(X_train_reshaped_nlp,X_train_reshaped_meta)


    print("\n\n THESE ARE THE LAYERS")
    for layer in model_meta_nlp_bert.layers:
      if(layer.name == 'input_idx' or layer.name == 'input_masks' or layer.name == 'tf_distil_bert_model'):
        layer.trainable = False
      
    model_meta_nlp_bert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
    model_meta_nlp_bert.summary()  
    
    #$$ this section is unique depending on the type of channel --starts
    """
    encoding the output labels
    """
    encoder = LabelEncoder()
    encoder.fit(df_Y_train_nlp)
    training_op_labels_encoded = encoder.transform(df_Y_train_nlp)
    test_op_labels_encoded = encoder.transform(df_Y_test_nlp)
    print("Output Labels\n-------------------")
    print(training_op_labels_encoded.shape, test_op_labels_encoded.shape)
    
    """
    converting the output labels to one-hot form
    """
    training_op_labels_onehot= np_utils.to_categorical(training_op_labels_encoded)
    test_op_labels_onehot = np_utils.to_categorical(test_op_labels_encoded)

    print(training_op_labels_onehot.shape, len(training_op_labels_onehot))
    print(test_op_labels_onehot.shape, len(test_op_labels_onehot))
    
    #$$ this section is unique depending on the type of channel -ends
    
    
    
    # ----------------------------------------------------
    # can save and read back for iterations
    # X_train_reshaped_nlp, X_test_reshaped_nlp
    # X_train_we, X_test_we
    # training_op_labels_onehot, test_op_labels_onehot
    # ----------------------------------------------------
    
    
    
    
    
    
    
    
    """
    $$$$ ---------------
    $$$$ CHANGE HERE _word_meta _word_nlp _nlp_meta _word_nlp_meta
    $$$$ ---------------
    """






    #change the input_
    #train data
   
    X_train_reshaped_meta = np.asarray(X_train_reshaped_meta, dtype='float32')
    model_meta_nlp_bert_train = model_meta_nlp_bert.fit(x=[X_train_reshaped_nlp,X_train_reshaped_meta, X_train_id,X_train_mask], y=training_op_labels_onehot,batch_size=BATCH_SIZE, epochs=EPOCHS,  validation_split=0.1)
    # load model if required 
    # compile model


    model_meta_nlp_bert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
    
    # evaluate model
    print(model_meta_nlp_bert.metrics_names)
    #change the input_

    #loss, binary_accuracy, categorical_accuracy = model_nlp_bert.evaluate(x=[X_test_reshaped_nlp,X_test_id,X_test_mask], y=test_op_labels_onehot,batch_size=BATCH_SIZE,verbose=1)
    
    #print(loss, binary_accuracy, categorical_accuracy)
    

    """
    predict the probabilty of output classes
    and pick the best one
    """
    #change the input_
    X_test_reshaped_meta = np.asarray(X_test_reshaped_meta, dtype='float32')
    prediction_vector = model_meta_nlp_bert.predict(x=[X_test_reshaped_nlp,X_test_reshaped_meta, X_test_id,X_test_mask], \
                           batch_size=BATCH_SIZE,\
                           verbose=1)
    predicted_classes = np.argmax(prediction_vector, axis=1)
    original_classes = np.argmax(test_op_labels_onehot, axis=1)
    accuracy = metrics.accuracy_score(original_classes, predicted_classes)

    print("ACCURACY:")
    print(accuracy)
    """
    # verification of correctness:
    total_correct = sum(original_classes == predicted_classes)
    print("Total number of correct predictions:",total_correct)
    print("Accuracy:",total_correct/len(test_op_labels_onehot))
    acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
    print('Overall accuracy: {} %'.format(acc*100))
    """
    conf_mat = confusion_matrix(predicted_classes, original_classes)
    print("\n", conf_mat, "\n")
    
    return predicted_classes, accuracy, conf_mat

# Main Method

In [27]:
"""
import tensorflow as tf
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf_config.allow_soft_placement = True
"""

import numpy as np 
import pandas as pd
from numpy import array

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D
from keras.layers.merge import concatenate
from keras.models import Model, model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import np_utils, to_categorical
from keras.engine.topology import Layer
from keras import backend as K

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

# checkpoint
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Attention

"""
Required for NLP model
"""
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()   

# set parameters for word embeddings
embed_size = 100 # how big is each word vector
vocab_size = 25000 # how many unique words to use (i.e num rows in embedding vector) max
input_length = 100 # max number of words in the input 

#set parameters for bilstm
BATCH_SIZE=32
EPOCHS=1

#EMBEDDING_FILE='glove.6B.100d.txt'    

file = open('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-nlp-bert-attn/bilstm_nlp_bert_attn_op.txt','w') #overwrites previous
file.close()

if __name__== "__main__":
    df_prediction = pd.DataFrame()
    df_accuracy =  pd.DataFrame()
    file = open('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-nlp-bert-attn/bilstm_meta_nlp_bert_attn_op.txt','a') #append mode 
    """
    Change the range before executing
    """
    for i in range(1,2):
        outputname = 'meta_meta_nlp_bert_attn'+ str(i)        
        predictions, acc, conf_matrix = execute_bilstm_meta_nlp_bert_channel(i)
        df_prediction[outputname] = predictions
        df_accuracy[i] = [acc]
        file.write("\nIteration:" + str(i) + "\nCategorical Accuracy:" + str(acc) + 
                    "\nConfusion Matrix:\n" + str(conf_matrix) + "\n\n")
        df_prediction.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-nlp-bert-attn/predictions_bilstm_meta_nlp_bert_attn_' + str(i) + '.csv')    
        df_accuracy.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-nlp-bert-attn/accuracy_bilstm_meta_nlp_bert_attn_' + str(i) + '.csv')    
    
    df_prediction.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-nlp-bert-attn/predictions_bilstm_meta_nlp_bert_attn.csv')    
    df_accuracy.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-nlp-bert-attn/accuracy_bilstm_nlp_meta_bert_attn.csv')    
    file.close()

Inside get_nlp_encodings
The size of training and test sets: (406, 10) (102, 10)
Inside generate_nlp_features
0
100
200
300
400
500
True
Inside encode_nlp_features
Size after split: (406, 123) (102, 123) (406, 1) (102, 1)
NLP data
------------------
Training set
------------------
[[[1 0 0 ... 1 1 0]]

 [[1 0 0 ... 1 1 0]]

 [[1 0 0 ... 1 0 0]]

 ...

 [[1 0 0 ... 1 0 0]]

 [[1 0 0 ... 1 0 0]]

 [[1 0 0 ... 1 0 0]]]
(406, 1, 123)
NLP data
------------------
Test set
------------------
[[[1 0 0 ... 1 0 0]]

 [[1 0 0 ... 1 0 0]]

 [[1 0 0 ... 1 0 0]]

 ...

 [[1 0 0 ... 1 0 0]]

 [[1 0 0 ... 1 0 0]]

 [[1 0 0 ... 1 0 0]]]
(102, 1, 123)
Output Labels
------------------
(406, 1) (102, 1)
Inside get_metadata_features
Meta data
------------------
Training set
------------------
[[[0.0 7.0 0 ... 1 2 2]]

 [[0.0 43.0 0 ... 1 11 1]]

 [[0.0 6.0 0 ... 1 1 3]]

 ...

 [[0.0 5.0 0 ... 10 2 2]]

 [[0.0 41.0 0 ... 1 1 3]]

 [[0.0 16.0 0 ... 12 12 3]]]
(406, 1, 8)
Meta data
------------------
Test se

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.




 THESE ARE THE LAYERS
Model: "functional_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_nlp (InputLayer)          [(None, 1, 123)]     0                                            
__________________________________________________________________________________________________
input_meta (InputLayer)         [(None, 1, 8)]       0                                            
__________________________________________________________________________________________________
input_idx (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 150)]        0                                            
______________________________________________________________

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


12/12 [==============================] - 7s 622ms/step - loss: 1.4127 - binary_accuracy: 0.7500 - categorical_accuracy: 0.3260 - val_loss: 1.4150 - val_binary_accuracy: 0.7500 - val_categorical_accuracy: 0.3171
[]
4/4 [==============================] - 0s 93ms/step
ACCURACY:
0.3137254901960784

 [[ 0  0  0  0]
 [24 17 11  3]
 [13 12 15  7]
 [ 0  0  0  0]] 

